In [74]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
from datetime import datetime

import requests
import json

# current date
date = datetime.now().strftime("%Y%m%d-%H%M")
print(date)

20221104-1537


In [75]:
# create function to export to csv
def export_csv(df, filename):
    df.to_csv(f'../../output/{date}-{filename}.csv', encoding='utf-8')
    print(f"Exported {filename}")

In [76]:
# keep columns
keep_cols = [
    'id',
    'utcDate',
    'status',
    'matchday',
    # 'season.id',
    # 'season.currentMatchday',
    # 'homeTeam.id',
    # 'homeTeam.name',
    'homeTeam.shortName',
    # 'awayTeam.id',
    # 'awayTeam.name',
    'awayTeam.shortName',
    # 'score.winner',
    'score.fullTime.home',
    'score.fullTime.away',
    # 'score.halfTime.home',
    # 'score.halfTime.away'
]

In [77]:
# All of premier league history since 1992-08-15
# uri = 'https://api.football-data.org/v2/competitions/PL/matches?dateFrom=1992-08-15'

uri = 'http://api.football-data.org/v4/competitions/PL/matches?status=FINISHED'
headers = { 'X-Auth-Token': '73015e4f2665414cb1fafe97e8f9971e' }

response = requests.get(uri, headers=headers)
data = response.json()
df = pd.json_normalize(data, record_path = ['matches'])
df = df[keep_cols]
df['utcDate'] = pd.to_datetime(df['utcDate'])
df['score.fullTime'] = df['score.fullTime.home'].astype(str) + '-' + df['score.fullTime.away'].astype(str)
print(df.shape)
df.head()


(126, 9)


,id,utcDate,status,matchday,homeTeam.shortName,awayTeam.shortName,score.fullTime.home,score.fullTime.away,score.fullTime
0,416384,2022-08-05 19:00:00+00:00,FINISHED,1,Crystal Palace,Arsenal,0,2,0-2
1,416383,2022-08-06 11:30:00+00:00,FINISHED,1,Fulham,Liverpool,2,2,2-2
2,416378,2022-08-06 14:00:00+00:00,FINISHED,1,Tottenham,Southampton,4,1,4-1
3,416379,2022-08-06 14:00:00+00:00,FINISHED,1,Newcastle,Nottingham,2,0,2-0
4,416381,2022-08-06 14:00:00+00:00,FINISHED,1,Leeds United,Wolverhampton,2,1,2-1


In [78]:
# pivot
df_table = df.pivot_table(index=['homeTeam.shortName'], columns=['awayTeam.shortName'], fill_value='.', values=['score.fullTime'], aggfunc='first')
# export to csv using function
export_csv(df_table, "table")

df_table.head()

Exported table


score.fullTime                                    \
awayTeam.shortName        Arsenal Aston Villa Bournemouth Brentford   
homeTeam.shortName                                                    
Arsenal                         .         2-1           .         .   
Aston Villa                     .           .           .       4-0   
Bournemouth                   0-3         2-0           .       0-0   
Brentford                     0-3           .           .         .   
Brighton Hove                   .           .           .         .   

                                                                        \
awayTeam.shortName Brighton Hove Chelsea Crystal Palace Everton Fulham   
homeTeam.shortName                                                       
Arsenal                        .       .              .       .    2-1   
Aston Villa                    .     0-2              .     2-1      .   
Bournemouth                    .       .              .       .      .   
Brentford                    2-0     0-0              .     1-1      .   
Brighton Hove                  .     4-1              .       .      .   

                                                                              \
awayTeam.shortName Leeds United Leicester City Liverpool Man City Man United   
homeTeam.shortName                                                             
Arsenal                       .            4-2       3-2        .          .   
Aston Villa                   .              .         .      1-1          .   
Bournemouth                   .            2-1         .        .          .   
Brentford                   5-2              .         .        .        4-0   
Brighton Hove               1-0            5-2         .        .          .   

                                                                        \
awayTeam.shortName Newcastle Nottingham Southampton Tottenham West Ham   
homeTeam.shortName                                                       
Arsenal                    .        5-0           .       3-1        .   
Aston Villa                .          .         1-0         .      0-1   
Bournemouth                .          .         0-1       2-3        .   
Brentford                  .          .           .         .        .   
Brighton Hove            0-0        0-0           .       0-1        .   

                                  
awayTeam.shortName Wolverhampton  
homeTeam.shortName                
Arsenal                        .  
Aston Villa                    .  
Bournemouth                  0-0  
Brentford                    1-1  
Brighton Hove                  .

## Create Matches DataFrame with a row for each team 

In [79]:
points_map = {
    'W': 3,
    'D': 1,
    'L': 0
}

def get_result(score, score_opp):
    if score == score_opp:
        return 'D'
    elif score > score_opp:
        return 'W'
    else:
        return 'L'

drop_cols = ['homeTeam.shortName','awayTeam.shortName', 'score.fullTime.home', 'score.fullTime.away']

# convert each match into two rows (one for each team)

df['H'] = df['homeTeam.shortName']
df['A'] = df['awayTeam.shortName']

df_matches = pd.melt(
    df,
    id_vars=keep_cols,
    value_vars=["H", "A"],
    var_name="home_away",
    value_name="team",
)
df_matches['opponent'] = np.where(
    df_matches['home_away'] == 'H', 
    df_matches['awayTeam.shortName'], 
    df_matches['homeTeam.shortName']
    )

# full time goals
df_matches["goals_scored"] = np.where(
    df_matches["team"] == df_matches["homeTeam.shortName"],
    df_matches["score.fullTime.home"],
    df_matches["score.fullTime.away"],
)
df_matches["goals_against"] = np.where(
    df_matches["team"] != df_matches["homeTeam.shortName"],
    df_matches["score.fullTime.home"],
    df_matches["score.fullTime.away"],
)
df_matches["result"] = np.vectorize(get_result)(
    df_matches["goals_scored"], df_matches["goals_against"]
)
df_matches["points"] = df_matches["result"].map(points_map)


# sort df_matches by id
df_matches = (df_matches
    .drop(drop_cols, axis=1)
    .sort_values(by=['utcDate', 'id'], ascending=True)
    .reset_index(drop=True)
    )

# export to csv using function
export_csv(df_matches, "matches")

df_matches.head(20)

Exported matches


,id,utcDate,status,matchday,home_away,team,opponent,goals_scored,goals_against,result,points
0,416384,2022-08-05 19:00:00+00:00,FINISHED,1,H,Crystal Palace,Arsenal,0,2,L,0
1,416384,2022-08-05 19:00:00+00:00,FINISHED,1,A,Arsenal,Crystal Palace,2,0,W,3
2,416383,2022-08-06 11:30:00+00:00,FINISHED,1,H,Fulham,Liverpool,2,2,D,1
3,416383,2022-08-06 11:30:00+00:00,FINISHED,1,A,Liverpool,Fulham,2,2,D,1
4,416378,2022-08-06 14:00:00+00:00,FINISHED,1,H,Tottenham,Southampton,4,1,W,3
5,416378,2022-08-06 14:00:00+00:00,FINISHED,1,A,Southampton,Tottenham,1,4,L,0
6,416379,2022-08-06 14:00:00+00:00,FINISHED,1,H,Newcastle,Nottingham,2,0,W,3
7,416379,2022-08-06 14:00:00+00:00,FINISHED,1,A,Nottingham,Newcastle,0,2,L,0
8,416381,2022-08-06 14:00:00+00:00,FINISHED,1,H,Leeds United,Wolverhampton,2,1,W,3
9,416381,2022-08-06 14:00:00+00:00,FINISHED,1,A,Wolverhampton,Leeds United,1,2,L,0


## Create league table

In [80]:
def league(x):
    results = {
        'Played': x['result'].count(),
        'Wins': (x['result'] == 'W').sum(),
        'Draws': (x['result'] == 'D').sum(),
        'Losses': (x['result'] == 'L').sum(),
        'Goals For': x['goals_scored'].sum(),
        'Goals Against': x['goals_against'].sum(),
        'Goal Difference': x['goals_scored'].sum() - x['goals_against'].sum(),
        'Points': x['points'].sum(),
        'form': ''.join(x['result'].tail(5).tolist()),
        # win %
        'win_%': '{:.0%}'.format((x['result'] == 'W').sum() / x['result'].count()),        
        # percentage of points from last 5 games
        'form_%': '{:.0%}'.format(x['points'].tail(5).sum() / 15),       
        # goals per game
        'gpg': '{:.2f}'.format(x['goals_scored'].sum() / x['result'].count()),
        # goals against per game
        'gapg': '{:.2f}'.format(x['goals_against'].sum() / x['result'].count()),
        # goal difference per game
        'gdpg': '{:.2f}'.format((x['goals_scored'].sum() - x['goals_against'].sum()) / x['result'].count()),
        # points per game
        'ppg': '{:.2f}'.format(x['points'].sum() / x['result'].count()),
        # maximum possible points remainig game for the seaon * 3
        'max_points': ((38 - x['result'].count()) * 3) + x['points'].sum(),
        # Expected points, based on points per game
        'expected_points': '{:.0f}'.format((x['points'].sum() / x['result'].count()) * (38 - x['result'].count()) + x['points'].sum()),

    }
    return pd.Series(results)

# apply league function
df_league = df_matches.groupby('team').apply(league)
# merge in form
# df_league = df_league.merge(df_form, left_index=True, right_index=True)
# Sort by points and then goal difference
df_league = df_league.sort_values(by=['Points', 'Goal Difference'], ascending=False)
# add column for position
df_league['Position'] = df_league['Points'].rank(ascending=False, method='first')
# reformat position to int
df_league['Position'] = df_league['Position'].astype(int)
# export to csv
# export to csv using function
export_csv(df_league, "league")

df_league.head(20)


Exported league


,Played,Wins,Draws,Losses,Goals For,Goals Against,Goal Difference,Points,form,win_%,form_%,gpg,gapg,gdpg,ppg,max_points,expected_points,Position
team,,,,,,,,,,,,,,,,,,
Arsenal,12,10,1,1,30,11,19,31,WWWDW,83%,87%,2.50,0.92,1.58,2.58,109,98,1
Man City,12,9,2,1,37,11,26,29,WWLWW,75%,80%,3.08,0.92,2.17,2.42,107,92,2
Tottenham,13,8,2,3,26,16,10,26,WWLLW,62%,60%,2.00,1.23,0.77,2.00,101,76,3
Newcastle,13,6,6,1,24,10,14,24,WDWWW,46%,87%,1.85,0.77,1.08,1.85,99,70,4
Man United,12,7,2,3,17,16,1,23,WDWDW,58%,73%,1.42,1.33,0.08,1.92,101,73,5
Chelsea,12,6,3,3,17,15,2,21,WWDDL,50%,53%,1.42,1.25,0.17,1.75,99,66,6
Fulham,13,5,4,4,22,22,0,19,LDWWD,38%,53%,1.69,1.69,0.00,1.46,94,56,7
Brighton Hove,12,5,3,4,19,15,4,18,LLDLW,42%,27%,1.58,1.25,0.33,1.50,96,57,8
Liverpool,12,4,4,4,23,15,8,16,LWWLL,33%,40%,1.92,1.25,0.67,1.33,94,51,9


In [81]:
# pivot df_match to show team results be match day
df_matchday = df_matches.pivot_table(index=['team'], columns=['matchday'], fill_value='.', values=['result'], aggfunc='first')



df_matchday.head(20)

result                                    
matchday           1  2  3  4  5  6  8  9  10 11 12 13 14
team                                                     
Arsenal             W  W  W  W  W  L  W  W  W  W  .  D  W
Aston Villa         L  W  L  L  L  D  W  D  D  L  L  W  L
Bournemouth         W  L  L  L  D  W  D  D  W  D  L  L  L
Brentford           D  W  L  D  D  W  L  D  L  W  D  L  D
Brighton Hove       W  D  W  W  L  W  .  D  L  L  D  L  W
Chelsea             W  D  L  W  L  W  .  W  W  W  D  D  L
Crystal Palace      L  D  W  L  D  D  .  L  W  D  W  L  W
Everton             L  L  D  D  D  D  W  W  L  L  L  W  D
Fulham              D  D  W  L  W  L  W  L  L  D  W  W  D
Leeds United        W  D  W  L  D  L  .  D  L  L  L  L  W
Leicester City      D  L  L  L  L  L  L  W  L  D  W  W  L
Liverpool           D  D  L  W  W  D  .  D  L  W  W  L  L
Man City            W  W  D  W  W  D  W  W  W  L  .  W  W
Man United          L  L  W  W  W  W  .  L  W  D  W  D  W
Newcastle           W  D  D  D  L  D  D  W  W  D  W  W  W
Nottingham          L  W  D  L  L  L  L  L  D  L  D  W  L
Southampton         L  D  W  L  W  L  L  L  L  D  W  D  L
Tottenham           W  D  W  W  D  W  W  L  W  W  L  L  W
West Ham            L  L  L  W  D  L  L  W  W  D  L  W  L
Wolverhampton       L  D  L  D  D  W  L  L  L  W  L  L  D